In [14]:

!pip install -U keras-tuner

     |████████████████████████████████| 61kB 3.6MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=5fc06573a8d4b4884543d20ae5665740db0b7a1ae3560ab664afc55bfefdf5bf
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=54b896e0cca8cde6ce063d063238c296d6a84a6b0f178e5502f585145ad57a03
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [15]:
import tensorflow as tf
from tensorflow import keras 
import numpy as np
print(tf.__version__)

2.2.0


In [0]:
#download fashion mnist datset from keras 

fashion_mnist = keras.datasets.fashion_mnist


In [9]:
fashion_mnist

(train_images, train_labels), (test_images, test_labels)= fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [0]:
# we are scaling the values to reduce the computation cost.

train_images = train_images/255
test_images = test_images/255

In [0]:
#we are reshaping our each examle in 4-Dimension 

train_images = train_images.reshape(len(train_images), 28,28, 1)

test_images = test_images.reshape(len(test_images), 28,28,1)


In [0]:
# lets define a funciton for building architecture of model

def build_model(hp):
  model = keras.Sequential([
          keras.layers.Conv2D(                   #a minimal, a maximal and a default value for the Float and the Int types are given
              filters = hp.Int('conv_1_filter', min_value = 32, max_value= 128, step= 16), #hp.choice is used for creating hyperparameter search space for integer value
                                                             #optionally, a step value, i.e the minimal step between two hyperparameter values
              kernel_size = hp.Choice('conv_1_kernel', values = [3,5]), #hp.choice is used to select any value from given set of value in this case either 3 or 5
              activation = 'relu',     
              input_shape = (28,28,1)
          ),
          keras.layers.Conv2D(
              filters = hp.Int('conv_2_filter', min_value = 32, max_value= 64, step= 16),
              kernel_size = hp.Choice('conv_2_kernel', values = [3,5]),
              activation = 'relu'
          ),
          keras.layers.Flatten(),
          keras.layers.Dense(
              units = hp.Int('dense_1_units', min_value = 32, max_value=64, step = 16),
              activation = 'relu'
          ),
          keras.layers.Dense(10, 'softmax')                            
  ])

  model.compile(optimizer = keras.optimizers.Adam(hp.Choice('learning_rate', values= [1e-2, 1e-3 ])),
                loss = 'sparse_categorical_crossentropy',
                metrics = ['accuracy'])
  
  return model

In [0]:

from tensorflow import keras
from kerastuner.engine.hyperparameters import HyperParameters
from kerastuner.tuners import RandomSearch, Hyperband

In [0]:
tuner_search = RandomSearch(build_model, 
                            objective= 'val_accuracy',
                            max_trials = 5, directory = 'output', project_name = 'Mnist Fashion' )
#max_trials variable represents the number of hyperparameter combinations that will be tested by the tuner,
# execution_per_trial variable is the number of models that should be built and fit for each trial for robustness purposes. Th

In [0]:
#search paramter starts training all the models

tuner_search.search(train_images, train_labels, epochs = 3, validation_split = 0.1)

INFO:tensorflow:Oracle triggered exit


In [0]:
# finds the best model

model = tuner_search.get_best_models(1)[0]


In [0]:
# all hyperparamter of the best models is given

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 48)        13872     
_________________________________________________________________
flatten (Flatten)            (None, 27648)             0         
_________________________________________________________________
dense (Dense)                (None, 48)                1327152   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                490       
Total params: 1,341,834
Trainable params: 1,341,834
Non-trainable params: 0
_________________________________________________________________


In [0]:
#best model is fit on the training set

model.fit(train_images, train_labels, epochs = 10, initial_epoch=3, validation_split = 0.15) 

Epoch 4/10
1594/1594 [==============================] - 6s 4ms/step - loss: 0.1609 - accuracy: 0.9407 - val_loss: 0.2232 - val_accuracy: 0.9236
Epoch 5/10
1594/1594 [==============================] - 6s 4ms/step - loss: 0.1204 - accuracy: 0.9557 - val_loss: 0.2474 - val_accuracy: 0.9188
Epoch 6/10
1594/1594 [==============================] - 6s 4ms/step - loss: 0.0868 - accuracy: 0.9678 - val_loss: 0.2581 - val_accuracy: 0.9212
Epoch 7/10
1594/1594 [==============================] - 6s 4ms/step - loss: 0.0645 - accuracy: 0.9766 - val_loss: 0.3136 - val_accuracy: 0.9211
Epoch 8/10
1594/1594 [==============================] - 6s 4ms/step - loss: 0.0496 - accuracy: 0.9825 - val_loss: 0.3610 - val_accuracy: 0.9203
Epoch 9/10
1594/1594 [==============================] - 6s 4ms/step - loss: 0.0384 - accuracy: 0.9861 - val_loss: 0.3934 - val_accuracy: 0.9137
Epoch 10/10
1594/1594 [==============================] - 6s 4ms/step - loss: 0.0313 - accuracy: 0.9891 - val_loss: 0.4095 - val_accuracy

In [0]:
#predicting test accuracy

score = model.evaluate(test_images, test_labels)
print('Test loss:', score[0])

print('Test accuracy:', score[1])

313/313 [==============================] - 1s 2ms/step - loss: 0.4791 - accuracy: 0.9069
Test loss: 0.4790755808353424
Test accuracy: 0.9068999886512756


**We are using Hyperband Instead of Randomsearch and building the model again to to improve accuracy**




#Hyperband is an optimized version of random search which uses early-stopping to speed up the hyperparameter tuning process. The main idea is to fit a large number of models for a small number of epochs and to only continue training for the models achieving the highest accuracy on the validation set. The max_epochs variable is the max number of epochs that a model can be trained for.

In [31]:

# Hyperband is initiated and parameters are filled into it

from kerastuner.tuners import Hyperband
#HYPERBAND_MAX_EPOCHS = 10
MAX_TRIALS = 5


tuner = Hyperband(
    build_model,
    max_epochs=4,
    objective='val_accuracy',
    directory='hyperband',
    project_name='cifar10'
)

INFO:tensorflow:Reloading Oracle from existing project hyperband/cifar10/oracle.json
INFO:tensorflow:Reloading Tuner from hyperband/cifar10/tuner0.json


In [32]:
# starts building model

tuner.search(train_images, train_labels, epochs = 3, validation_split = 0.12)

INFO:tensorflow:Oracle triggered exit


In [24]:
model_2 = tuner.get_best_models(1)[0]


In [25]:
model_2.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 48)        480       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 64)        76864     
_________________________________________________________________
flatten (Flatten)            (None, 30976)             0         
_________________________________________________________________
dense (Dense)                (None, 48)                1486896   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                490       
Total params: 1,564,730
Trainable params: 1,564,730
Non-trainable params: 0
_________________________________________________________________


In [26]:
#predicting test accuracy
model_2.fit(train_images, train_labels, epochs = 10, initial_epoch=3, validation_split = 0.15) 


Epoch 4/10
1594/1594 [==============================] - 9s 5ms/step - loss: 0.3496 - accuracy: 0.8707 - val_loss: 0.4148 - val_accuracy: 0.8580
Epoch 5/10
1594/1594 [==============================] - 9s 5ms/step - loss: 0.3275 - accuracy: 0.8771 - val_loss: 0.3569 - val_accuracy: 0.8707
Epoch 6/10
1594/1594 [==============================] - 8s 5ms/step - loss: 0.3167 - accuracy: 0.8809 - val_loss: 0.3848 - val_accuracy: 0.8637
Epoch 7/10
1594/1594 [==============================] - 8s 5ms/step - loss: 0.3016 - accuracy: 0.8879 - val_loss: 0.4021 - val_accuracy: 0.8559
Epoch 8/10
1594/1594 [==============================] - 8s 5ms/step - loss: 0.2926 - accuracy: 0.8919 - val_loss: 0.4465 - val_accuracy: 0.8633
Epoch 9/10
1594/1594 [==============================] - 8s 5ms/step - loss: 0.2809 - accuracy: 0.8957 - val_loss: 0.4208 - val_accuracy: 0.8651
Epoch 10/10
1594/1594 [==============================] - 8s 5ms/step - loss: 0.2683 - accuracy: 0.9019 - val_loss: 0.4001 - val_accuracy

In [28]:
score = model_2.evaluate(test_images, test_labels)
print('Test loss:', score[0])

print('Test accuracy:', score[1])

313/313 [==============================] - 1s 3ms/step - loss: 0.4424 - accuracy: 0.8626
Test loss: 0.4423786997795105
Test accuracy: 0.8626000285148621
